In [1]:
# Extract 3-hourly 500 hPa Omega from regridded vertical velocity (wa) data of UM zoom 6 model
# Extract 6-hourly 500 hPa Omega from regridded vertical velocity (wa) data of ICON zoom 6 model
import xarray as xr
from pathlib import Path
import easygems.healpix as egh
import healpy as hp
import numpy as np

p = 500 # hPa; level of interest
data = "/g/data/qx55/uk_node/glm.n2560_RAL3p3/data.healpix.PT3H.z6.zarr" #PT1H is hourly data
data = "/g/data/qx55/germany_node/d3hp003.zarr/PT6H_point_z6_atm.zarr" #PT6H is 6-hourly data
ds = xr.open_zarr(data)

#w = ds['wa'].sel(pressure=p)
#t = ds['ta'].sel(pressure=p)
w = ds['wa'].sel(pressure=p*100)
t = ds['ta'].sel(pressure=p*100)

def get_nn_lon_lat_index(nside, lons, lats):
    """
    nside: integer, power of 2. The return of hp.get_nside()
    lons: uniques values of longitudes
    lats: uniques values of latitudes
    returns: array with the HEALPix cells that are closest to the lon/lat grid
    """
    lons2, lats2 = np.meshgrid(lons, lats)
    return xr.DataArray(
        hp.ang2pix(nside, lons2, lats2, nest = True, lonlat = True),
        coords=[("latitude", lats), ("longitude", lons)],
    )

In [2]:
from metpy.calc import vertical_velocity_pressure
from metpy.units import units

this_nside = hp.get_nside(w)
res = 1  # resolution
#MC lon: 100 - 149E, lat: 13S to 13N
lon = np.arange(100, 149 + res, res)
lat = np.arange(-13, 13 + res, res)

cells = get_nn_lon_lat_index(this_nside, lon, lat) 
w_regrid = w.isel(cell=cells)
t_regrid = t.isel(cell=cells)

omega = vertical_velocity_pressure(w_regrid * units('m/s'), 500 * units.hPa, t_regrid * units.degK)

In [4]:
import pandas as pd
# Create full date range from 2020-03-01 to 2021-02-28
#full_range = pd.date_range(start="2020-03-01", end="2021-02-28", freq="3h")
full_range = pd.date_range(start="2020-03-01", end="2021-02-28", freq="6h")
date_str_list = full_range.strftime("%Y%m%d %H:%M:%S").tolist()

w500 = omega.sel(time=date_str_list).mean(dim=("latitude","longitude"))
w500 = w500.metpy.dequantify()

In [5]:
dir = '/scratch/nf33/hk25_LSP/UK/MC/'
dir = '/scratch/nf33/hk25_LSP/Germany/Darwin/'
fname = 'w500_3hr.nc'
fname = 'w500_6hr.nc'

w500.to_netcdf(dir+fname)
print(f"Saved to ",dir+fname)

Saved to  /scratch/nf33/hk25_LSP/Germany/Darwin/w500_6hr.nc
